In [11]:
import pandas as pd
from datetime import datetime

In [8]:
flights = pd.read_csv('flights.csv', delimiter=';')

## Pre-processing

In [12]:
flights

,source,destination,departure,arrival,flight_number,price,bags_allowed,bag_price
0,USM,HKT,2017-02-11T06:25:00,2017-02-11T07:25:00,PV404,24,1,9
1,USM,HKT,2017-02-12T12:15:00,2017-02-12T13:15:00,PV755,23,2,9
2,USM,HKT,2017-02-12T21:15:00,2017-02-12T22:15:00,PV729,25,1,14
3,USM,HKT,2017-02-11T14:50:00,2017-02-11T15:50:00,PV966,21,1,17
4,USM,HKT,2017-02-12T00:35:00,2017-02-12T01:35:00,PV398,24,1,14
5,USM,HKT,2017-02-12T05:15:00,2017-02-12T06:15:00,PV870,19,1,13
6,USM,HKT,2017-02-12T10:00:00,2017-02-12T11:00:00,PV320,22,1,18
7,USM,HKT,2017-02-12T13:40:00,2017-02-12T14:40:00,PV540,26,2,13
8,USM,HKT,2017-02-12T09:30:00,2017-02-12T10:30:00,PV290,19,2,8
9,USM,HKT,2017-02-11T02:40:00,2017-02-11T03:40:00,PV876,25,2,16
